In [3]:
from llama_cpp import Llama
import json
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# 1. GGUF modeli yükle
llm = Llama(
    model_path="mistral-7b-instruct-v0.2-turkish.Q3_K_M.gguf",
    n_ctx=1024,
    n_threads=4,
    n_gpu_layers=0,
    verbose=False
)

# 2. Tahmin fonksiyonu
def analyze_sentiment(text):
    prompt = f"""Aşağıdaki müşteri yorumunun duygu analizini yap. Sadece şunlardan birini yaz: Pozitif, Negatif, Nötr.

Yorum: "{text}"
Cevap:"""
    output = llm(prompt, max_tokens=32)
    return output["choices"][0]["text"].strip().lower()

# 3. Etiket dönüşüm tablosu
etiket_map = {
    "pozitif": 1,
    "olumlu": 1,
    "nötr": 0,
    "neutral": 0,
    "negatif": 2,
    "olumsuz": 2
}


llama_context: n_ctx_per_seq (1024) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [4]:
with open("etiketli_yorumlar_mistral.json", "r", encoding="utf-8") as f:
    etiketli = json.load(f)["reviews"]

# Gerçek etiketli yorumları al
yorumlar_test = [y for y in etiketli if isinstance(y.get("label"), int)]

# Tahminler
y_true, y_pred = [], []

for yorum in tqdm(yorumlar_test[:20]):  # test için ilk 20
    yorum_text = yorum["comment"].strip()
    tahmin = analyze_sentiment(yorum_text)
    tahmin_encoded = etiket_map.get(tahmin, -1)

    if tahmin_encoded != -1:
        y_pred.append(tahmin_encoded)
        y_true.append(yorum["label"])  # Artık direkt alabiliyoruz

# Değerlendirme
print("🔢 Tahmin Sayısı:", len(y_pred))
print("🎯 Gerçek Etiket Sayısı:", len(y_true))

if len(set(y_true)) > 0:
    print("✅ Accuracy:", accuracy_score(y_true, y_pred))
    print("📊 Classification Report:")
    print(classification_report(
        y_true,
        y_pred,
        target_names=["Nötr", "Pozitif", "Negatif"],
        labels=[0, 1, 2]
    ))
else:
    print("⚠️ Uyarı: Gerçek etiketlerde sınıf bulunamadı.")


100%|██████████| 20/20 [05:10<00:00, 15.51s/it]

🔢 Tahmin Sayısı: 11
🎯 Gerçek Etiket Sayısı: 11
✅ Accuracy: 0.9090909090909091
📊 Classification Report:
              precision    recall  f1-score   support

        Nötr       0.00      0.00      0.00         1
     Pozitif       0.91      1.00      0.95        10
     Negatif       0.00      0.00      0.00         0

    accuracy                           0.91        11
   macro avg       0.30      0.33      0.32        11
weighted avg       0.83      0.91      0.87        11




c:\Users\ŞEVVAL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ŞEVVAL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ŞEVVAL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

In [5]:
for yt, yp, y in zip(y_true, y_pred, yorumlar_test[:20]):
    if yt != yp:
        print(f"\n❌ Yanlış Tahmin:\nYorum: {y['comment']}\nBeklenen: {yt} | Tahmin: {yp}")



❌ Yanlış Tahmin:
Yorum: iki yıl geçmesine ragmen bir sorun yaşamadım hafif ve kullanışlı
Beklenen: 0 | Tahmin: 1


In [8]:
# Örnek yorum listesi
ornek_yorumlar = [
    "Ürün beklediğimden çok daha iyi geldi, paketleme harikaydı.",
    "Berbat bir deneyimdi, bir daha asla sipariş vermem.",
    "Ne iyi ne kötü, idare eder.",
    "Kargo biraz gecikti ama müşteri hizmetleri yardımcı oldu.",
    "Rengi fotoğraftakinden çok farklı geldi, hayal kırıklığı.",
    "Ürün fena değil fiyat performans",
    "Hiç beğenmedim selesi sert kalitesiz",
    "Ürünün kalitesi çok iyi",
    "Daha iyi olabilirdi",
    "Çok güzel ürün dağdan aşağı kaydım"
]

# Tahmin et
for yorum in ornek_yorumlar:
    sonuc = analyze_sentiment(yorum)
    print(f"\n📝 Yorum: {yorum}")
    print(f"📊 Tahmin: {sonuc}")



📝 Yorum: Ürün beklediğimden çok daha iyi geldi, paketleme harikaydı.
📊 Tahmin: pozitif.

📝 Yorum: Berbat bir deneyimdi, bir daha asla sipariş vermem.
📊 Tahmin: olumsuz.

📝 Yorum: Ne iyi ne kötü, idare eder.
📊 Tahmin: doğal.

📝 Yorum: Kargo biraz gecikti ama müşteri hizmetleri yardımcı oldu.
📊 Tahmin: doğal.

📝 Yorum: Rengi fotoğraftakinden çok farklı geldi, hayal kırıklığı.
📊 Tahmin: olumsuz

📝 Yorum: Ürün fena değil fiyat performans
📊 Tahmin: doğal.

📝 Yorum: Hiç beğenmedim selesi sert kalitesiz
📊 Tahmin: negatif.

📝 Yorum: Ürünün kalitesi çok iyi
📊 Tahmin: pozitif

📝 Yorum: Daha iyi olabilirdi
📊 Tahmin: doğal.

📝 Yorum: Çok güzel ürün dağdan aşağı kaydım
📊 Tahmin: pozitif.
